In [47]:
import pandas as pd
from datetime import datetime, timedelta

def obtener_festivos_mexico(año):
    """
    Retorna un set de fechas festivas para México en un año dado
    """
    festivos = set()

    # Festivos de fecha fija
    festivos.add(datetime(año, 1, 1))   # Año Nuevo
    festivos.add(datetime(año, 5, 1))   # Día del Trabajo
    festivos.add(datetime(año, 9, 16))  # Día de la Independencia
    festivos.add(datetime(año, 12, 25)) # Navidad

    # Festivos móviles (se mueven al primer lunes)
    # Día de la Constitución (primer lunes de febrero)
    primer_dia_feb = datetime(año, 2, 1)
    dias_hasta_lunes = (7 - primer_dia_feb.weekday()) % 7
    if dias_hasta_lunes == 0:
        constitucion = primer_dia_feb
    else:
        constitucion = primer_dia_feb + timedelta(days=dias_hasta_lunes)
    festivos.add(constitucion)

    # Natalicio de Benito Juárez (tercer lunes de marzo)
    primer_dia_mar = datetime(año, 3, 1)
    dias_hasta_lunes = (7 - primer_dia_mar.weekday()) % 7
    if dias_hasta_lunes == 0:
        primer_lunes = primer_dia_mar
    else:
        primer_lunes = primer_dia_mar + timedelta(days=dias_hasta_lunes)
    benito_juarez = primer_lunes + timedelta(weeks=2)
    festivos.add(benito_juarez)

    # Día de la Revolución (tercer lunes de noviembre)
    primer_dia_nov = datetime(año, 11, 1)
    dias_hasta_lunes = (7 - primer_dia_nov.weekday()) % 7
    if dias_hasta_lunes == 0:
        primer_lunes = primer_dia_nov
    else:
        primer_lunes = primer_dia_nov + timedelta(days=dias_hasta_lunes)
    revolucion = primer_lunes + timedelta(weeks=2)
    festivos.add(revolucion)

    # Festivos adicionales importantes (no oficiales pero ampliamente celebrados)
    festivos.add(datetime(año, 2, 14))  # Día del Amor y la Amistad
    festivos.add(datetime(año, 5, 10))  # Día de las Madres
    festivos.add(datetime(año, 11, 2))  # Día de Muertos
    festivos.add(datetime(año, 12, 12)) # Día de la Virgen de Guadalupe

    return festivos


def generar_dim_tiempo(fecha_inicio, fecha_fin, archivo_salida='dim_tiempo.csv'):
    """
    Genera un archivo CSV con dimensión de tiempo

    Parámetros:
    - fecha_inicio: str o datetime, formato 'YYYY-MM-DD'
    - fecha_fin: str o datetime, formato 'YYYY-MM-DD'
    - archivo_salida: str, nombre del archivo CSV a generar
    """

    # Convertir a datetime si son strings
    if isinstance(fecha_inicio, str):
        fecha_inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    if isinstance(fecha_fin, str):
        fecha_fin = datetime.strptime(fecha_fin, '%Y-%m-%d')

    # Generar rango de fechas
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

    # Obtener todos los festivos para los años en el rango
    años_en_rango = range(fecha_inicio.year, fecha_fin.year + 1)
    festivos_totales = set()
    for año in años_en_rango:
        festivos_totales.update(obtener_festivos_mexico(año))

    print(f"✓ Festivos cargados: {len(festivos_totales)} días festivos")

    # Diccionarios para nombres en español
    nombres_meses = {
        1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril',
        5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto',
        9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
    }

    nombres_dias = {
        1: 'lunes', 2: 'martes', 3: 'miércoles', 4: 'jueves',
        5: 'viernes', 6: 'sábado', 7: 'domingo'
    }

    # Crear lista para almacenar los datos
    datos = []

    for idx, fecha in enumerate(fechas, start=1):
        # Convertir pandas Timestamp a datetime
        fecha_dt = fecha.to_pydatetime()

        # Calcular atributos
        año = fecha.year
        mes = fecha.month
        dia_año = fecha.dayofyear
        dia_mes = fecha.day
        dia_semana = fecha.dayofweek + 1  # 1=Lunes, 7=Domingo

        # Calcular semestre (1 o 2)
        semestre = 1 if mes <= 6 else 2

        # Calcular trimestre (1, 2, 3, o 4)
        trimestre = (mes - 1) // 3 + 1

        # Calcular quincena del mes (1 o 2)
        quincena_mes = 1 if dia_mes <= 15 else 2

        # Calcular quincena del año (1-24)
        quincena_anio = (mes - 1) * 2 + quincena_mes

        # Calcular semana del año (1-52/53)
        semana_del_anio = ((dia_año - 1) // 7) + 1

        # Calcular semana del mes (1-4 o 5)
        semana_del_mes = (dia_mes - 1) // 7 + 1

        # Verificar si es festivo
        fecha_sin_hora = datetime(año, mes, dia_mes)
        dia_festivo = 1 if fecha_sin_hora in festivos_totales else 0

        # Calcular día fin de semana (sábado o domingo)
        dia_fin_semana = 1 if dia_semana in [6, 7] else 0

        # Crear ID con formato t001, t002, etc.
        id_tiempo = f"t{idx:03d}"

        # Obtener nombres de mes y día
        nombre_mes = nombres_meses[mes]
        nombre_dia = nombres_dias[dia_semana]

        # Agregar registro
        datos.append({
            'id_tiempo': id_tiempo,
            'fecha': fecha.strftime('%Y-%m-%d'),
            'año': año,
            'semestre': semestre,
            'trimestre': trimestre,
            'mes': mes,
            'nombre_mes': nombre_mes,
            'quincena_mes': quincena_mes,
            'quincena_anio': quincena_anio,
            'semana_del_anio': semana_del_anio,
            'semana_del_mes': semana_del_mes,
            'dia_del_año': dia_año,
            'dia_del_mes': dia_mes,
            'dia_de_semana': dia_semana,
            'nombre_dia': nombre_dia,
            'dia_festivo': dia_festivo,
            'dia_fin_de_semana': dia_fin_semana
        })

    # Crear DataFrame
    df = pd.DataFrame(datos)

    # Guardar CSV
    df.to_csv(archivo_salida, index=False)
    print(f"✓ Archivo '{archivo_salida}' generado exitosamente")
    print(f"✓ Total de registros: {len(df)}")
    print(f"✓ Rango: {df['fecha'].min()} a {df['fecha'].max()}")
    print(f"✓ Días festivos encontrados: {df['dia_festivo'].sum()}")

    return df


# Ejemplo de uso
if __name__ == "__main__":
    # Generar datos para 2023 y 2024
    df = generar_dim_tiempo('2023-01-01', '2024-12-31')

    # Mostrar primeros registros
    print("\nPrimeros registros:")
    print(df.head(10))

    # Mostrar días festivos
    print("\nDías festivos en el rango:")
    festivos_df = df[df['dia_festivo'] == 1][['fecha', 'nombre_dia', 'nombre_mes']]
    print(festivos_df.to_string(index=False))

    # Leer el CSV generado
    df_leido = pd.read_csv('new_csv/dim_tiempo.csv')
    print("\n✓ CSV leído correctamente")
    print(f"Columnas: {list(df_leido.columns)}")

✓ Festivos cargados: 22 días festivos
✓ Archivo 'dim_tiempo.csv' generado exitosamente
✓ Total de registros: 731
✓ Rango: 2023-01-01 a 2024-12-31
✓ Días festivos encontrados: 22

Primeros registros:
  id_tiempo       fecha   año  semestre  trimestre  mes nombre_mes  \
0      t001  2023-01-01  2023         1          1    1      enero   
1      t002  2023-01-02  2023         1          1    1      enero   
2      t003  2023-01-03  2023         1          1    1      enero   
3      t004  2023-01-04  2023         1          1    1      enero   
4      t005  2023-01-05  2023         1          1    1      enero   
5      t006  2023-01-06  2023         1          1    1      enero   
6      t007  2023-01-07  2023         1          1    1      enero   
7      t008  2023-01-08  2023         1          1    1      enero   
8      t009  2023-01-09  2023         1          1    1      enero   
9      t010  2023-01-10  2023         1          1    1      enero   

   quincena_mes  quincena_anio

In [51]:
df_leido[df_leido["fecha"]=="2023-12-25"]

,id_tiempo,fecha,año,semestre,trimestre,mes,nombre_mes,quincena_mes,quincena_anio,semana_del_anio,semana_del_mes,dia_del_año,dia_del_mes,dia_de_semana,nombre_dia,dia_festivo,dia_fin_de_semana
358,t359,2023-12-25,2023,2,4,12,diciembre,2,24,52,4,359,25,1,lunes,1,0


In [43]:

df_leido.columns

Index(['id_tiempo', 'fecha', 'año', 'semestre', 'trimestre', 'mes',
       'nombre_mes', 'quincena_mes', 'quincena_anio', 'semana_del_anio',
       'semana_del_mes', 'dia_del_año', 'dia_del_mes', 'dia_de_semana',
       'nombre_dia', 'dia_festivo', 'dia_fin_de_semana'],
      dtype='object')